# Multi Language Detection

## Load Dataset from Hugging Face

In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset
import pandas as pd

In [3]:
dataset = load_dataset("papluca/language-identification", "default")
trainingData = dataset['train']

df = trainingData.to_pandas()
df

,labels,text
0,pt,"os chefes de defesa da estónia, letónia, lituâ..."
1,bg,размерът на хоризонталната мрежа може да бъде ...
2,zh,很好，以前从不去评价，不知道浪费了多少积分，现在知道积分可以换钱，就要好好评价了，后来我就把...
3,th,สำหรับ ของเก่า ที่ จริงจัง ลอง honeychurch ...
4,ru,Он увеличил давление .
...,...,...
69995,ja,本格的なゲーミングヘッドホンでした。 今まで使ってた1万円するパナソニックのヘッドホンは何だ...
69996,el,"Ναι , ξέρω ένα που είναι ακόμα έτσι , αλλά αυτ..."
69997,ur,اور مجھے اس ملک کے بارے میں معلوم نہیں ہے کہ گ...
69998,es,Se me rompió uno al sacarlo del cargador. Cali...


## Data Pre-Processing

### Clean Data
For this model, we are going to strip out Chinese, Korean, and Japanese becuase the encoding on those characters seems to be very difficult to train the model on. The final library will still be capable of identifying these languages by simply checking if they contain characters of that origin. We will also be cleaning up some of the text to remove numbers.

In [4]:
!pip install neattext

In [5]:
import neattext as nt
import neattext.functions as nfx

In [6]:
pd.options.display.max_colwidth=999

# Drop Chinese Rows
df = df[df.labels != 'zh']
# Drop Korean Rows
df = df[df.labels != 'ko']
# Drop Japanese Rows
df = df[df.labels != 'ja']
# Remove numbers
df['text'] = df['text'].apply(nfx.remove_numbers)

df

,labels,text
0,pt,"os chefes de defesa da estónia, letónia, lituânia, alemanha, itália, espanha e eslováquia assinarão o acordo para fornecer pessoal e financiamento para o centro."
1,bg,размерът на хоризонталната мрежа може да бъде по реда на няколко километра ( km ) за на симулация до около km за на симулация .
3,th,สำหรับ ของเก่า ที่ จริงจัง ลอง honeychurch ของเก่า ที่ ไม่ สำหรับ เฟอร์นิเจอร์ และ เงิน ไท ร้อง บริษัท ที่ สำหรับ ลาย คราม
4,ru,Он увеличил давление .
5,pl,"S Jak sobie życzysz: Widzisz, jak Hitler zabija Żydów?"
...,...,...
69994,ar,نعم انت ايضا مع السلامة مع السلامة
69996,el,"Ναι , ξέρω ένα που είναι ακόμα έτσι , αλλά αυτό είναι ότι είμαι κάτω σε ένα φίλο που κρατάει αυτή τη θέση ακόμα είμαι η μόνη στο σπίτι αυτή τη στιγμή και έχω τα παιδιά να φωνάζουν για την προσοχή μου τώρα , οπότε καταλαβαίνω τι . Αυτό είναι σαν"
69997,ur,اور مجھے اس ملک کے بارے میں معلوم نہیں ہے کہ گزشتہ سال کے دوران ، میں گزشتہ سال کے بارے میں نہیں جانتا تھا ۔
69998,es,"Se me rompió uno al sacarlo del cargador. Calidad nefasta para el precio que tiene, no lo recomiendo. Que pesaos con las palabras que faltan para la opinión por Dios."


### Restructure Data for Multi-Label Classification
Since we want this model to recognize every language in a string, we are going to be using Multi-Label Classification. This data is currently setup for Multi-Class Classification, where one input will be assigned one output. We will want it to look something like this:

| Text                                              | en | es | fr | hi |
| ------------------------------------------------- | -- | -- | -- | -- |
| test an english phrase                            |  1 |  0 |  0 |  0 |
| probar una frase en ingles                        |  0 |  1 |  0 |  0 |
| test an english phrase probar una frase en ingles |  1 |  1 |  0 |  0 |

In [7]:
import numpy as np

In [8]:
# Create a set of unique labels to represent all the languages in the dataset
unique_labels = df['labels'].unique()
num_rows = len(df)
num_cols = len(unique_labels) + 1  
reformatted_data = np.zeros((num_rows, num_cols), dtype=object)

reformatted_data[:, 0] = df['text']

# Fill the boolean values for labels
for i, label in enumerate(unique_labels):
    reformatted_data[:, i+1] = (df['labels'] == label)

# Create a DataFrame from the NumPy array
reformatted_df = pd.DataFrame(reformatted_data, columns=['text'] + list(unique_labels))

reformatted_df

,text,pt,bg,th,ru,pl,ur,sw,tr,es,ar,it,hi,de,el,nl,fr,vi,en
0,"os chefes de defesa da estónia, letónia, lituânia, alemanha, itália, espanha e eslováquia assinarão o acordo para fornecer pessoal e financiamento para o centro.",True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,размерът на хоризонталната мрежа може да бъде по реда на няколко километра ( km ) за на симулация до около km за на симулация .,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,สำหรับ ของเก่า ที่ จริงจัง ลอง honeychurch ของเก่า ที่ ไม่ สำหรับ เฟอร์นิเจอร์ และ เงิน ไท ร้อง บริษัท ที่ สำหรับ ลาย คราม,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,Он увеличил давление .,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,"S Jak sobie życzysz: Widzisz, jak Hitler zabija Żydów?",False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62995,نعم انت ايضا مع السلامة مع السلامة,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
62996,"Ναι , ξέρω ένα που είναι ακόμα έτσι , αλλά αυτό είναι ότι είμαι κάτω σε ένα φίλο που κρατάει αυτή τη θέση ακόμα είμαι η μόνη στο σπίτι αυτή τη στιγμή και έχω τα παιδιά να φωνάζουν για την προσοχή μου τώρα , οπότε καταλαβαίνω τι . Αυτό είναι σαν",False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
62997,اور مجھے اس ملک کے بارے میں معلوم نہیں ہے کہ گزشتہ سال کے دوران ، میں گزشتہ سال کے بارے میں نہیں جانتا تھا ۔,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
62998,"Se me rompió uno al sacarlo del cargador. Calidad nefasta para el precio que tiene, no lo recomiendo. Que pesaos con las palabras que faltan para la opinión por Dios.",False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


### Create Data for Multi-Language Recognition
The current dataset provides only one language per item in the text column. To train a model fo multi-language recognition, we need to create some new data that has texts with multiple languages and the labels marked accordingly. To do this, we will systematically add rows to the data that contain another random language and apply the other language label to the row.

In [51]:
pd.options.display.max_colwidth=999

lang1 = reformatted_df.sample(n=50000)
lang2 = reformatted_df.sample(n=50000)
lang3 = reformatted_df.sample(n=50000)

# Generate rows with 2 languages
data_rows = [
    [lang1['text'].iloc[index] + ' ' + lang2['text'].iloc[index]] +
    (lang1.iloc[index, 1:] | lang2.iloc[index, 1:]).astype(bool).tolist()
    for index in range(len(lang1))
]

new_data = pd.DataFrame(data_rows, columns=['text'] + list(unique_labels))

# Generate rows with 3 languages
data_rows = [
    [new_data['text'].iloc[index] + ' ' + lang3['text'].iloc[index]] +
    (new_data.iloc[index, 1:] | lang3.iloc[index, 1:]).astype(bool).tolist()
    for index in range(len(new_data))
]

new_data2 = pd.DataFrame(data_rows, columns=['text'] + list(unique_labels))
new_data2

,text,pt,bg,th,ru,pl,ur,sw,tr,es,ar,it,hi,de,el,nl,fr,vi,en
0,"Yeye shivered . यह एक सुंदर संरचना है , अपने porticoed बरामदे , वीं शताब ् दी के चित ् रों और इतालवी संगमरमर के फर ् श को बनाए रखें . This product was nice bt the color doesn't change and it doesn't show too well. The images online are misleading. Other then that it's a great doodle pad",False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True
1,"उदाहरण के लिए , ब ् याज- ( प ् रतिभूति के बाजार की कीमतों , जैसे राजकोष प ् रतिभूति , fluctuate inversely बाजार ब ् याज दरों के साथ । Najważniejsze punkty debaty wiceprzewodniczącego کچھ ادارے کے لیے ڈیزائن کردہ ، ڈیزائن کردہ مواد کا جائزہ لینے کے لیے تیار ہے .",False,False,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False
2,"Gatto bianco seduto all'interno di un lavandino. Waksal, in una lettera alla corte, ha detto: ""Ho fatto a pezzi la mia famiglia. des capteurs a une sensibilité qui n'est pas réglable (continue de se déclencher à tout moment). Les autres fonctionnent correctement.",False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False
3,"A mi hijo de años y medio le ENCANTA, LAS PIEZAS SON BASTANTE GORDITAS y eso lo convierte en muy resistente. विशेष रूप से शाम और सप ् ताहांत में , जब आप cubans और विदेशियों का एक हर ् षोल ् लास मिश ् रण का सामना करेंगे , द ् वीप का casas दे ला trova वास ् तव में झूला । Uh-huh vâng à không phải là bất thường những gì thực sự tồi tệ ở đây trong mùa đông là khi chúng tôi được ơ băng trên các con đường và nhiệt độ không được ở trên lạnh cóng không có các thiết bị xóa băng",False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,True,False
4,ء میں پرتگال نے یورپی معیشت میں شمولیت اختیار کی ۔ u.s. ( یورپی یونین ) Practically see through. Not what I expected. ء میں گفتگو کے طور پر ، یہ موجودہ اثاثوں میں سے ایک ہے جو کہ موجودہ سرمایہ کاری ، سرمایہ کاری اور خدمات کو بڑھانے میں مدد کرتا ہے ۔,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,Parfait pour mon Acer Aspire ! รับ อะไหล่ ที่ มี ช่องว่าง ใหญ่ ใน ตัว พวกเขา اور میں خیال کرتا ہوں کہ کینیڈی کے بارے میں ایک ایسا خیال ہے کہ میرے خیال میں ایک ایسا لگتا ہے کہ اصل بات یہ ہے ، یا آپ جانتے ہیں کہ آپ اس طرح کی دلچسپی رکھتے ہیں ۔,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
49996,"روندوا دیا گیا ، اس نے گھوڑے پر قبضہ کر لیا ۔ Przetrzymywanych jest około więźniów z krajów, w tym Kanady, wielu schwytanych podczas wojny z siecią Al-Kaida w Afganistanie. La Russia ha ratificato la versione modificata, ma gli Stati Uniti e gli altri membri della NATO si sono rifiutati di farlo fino a quando il governo russo non ritirerà le truppe dalle ex repubbliche sovietiche della Moldavia e della Georgia.",False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False,False
49997,правни услуги корпорация v. Een groep mensen in boten die naar de wal racen. L'Australia a istituire la più grande riserva marina del mondo,False,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False
49998,"Yol kenarı çöp toplamak için g- Zirvelerde ' i skip . लेकिन मुझे आज रात एक क ् लास मिल गया है इसलिए मैं इन ् हें रास ् ते से बाहर करना चाहता था Heute kam mein Überraschungspaket, wie erwartet. Ich will mich für diesen Preis nicht beklagen, aber es war viel Plunder enthalten, vieles für Weihnachten aber auch einiges, das mir gefallen hat wie zwei Lichterketten und viele Geschenkbänder. Auch die Servietten sind ganz süß, allerdings wie gesagt für Weihnachten. Ich habe euch mal ein Foto hochgeladen, damit ihr einen Überblick kriegt - wie gesagt, das Paket ist ganz ok, würde jedoch nicht wieder bestellen.",False,False,False,False,Fals

### Combine New Data with Existing

In [52]:
pd.options.display.max_colwidth=999

multi_df = pd.concat([reformatted_df, new_data, new_data2])
# transform to booleans floats for model training purposes
multi_df[unique_labels] = multi_df[unique_labels].astype(int)
multi_df

,text,pt,bg,th,ru,pl,ur,sw,tr,es,ar,it,hi,de,el,nl,fr,vi,en
0,"os chefes de defesa da estónia, letónia, lituânia, alemanha, itália, espanha e eslováquia assinarão o acordo para fornecer pessoal e financiamento para o centro.",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,размерът на хоризонталната мрежа може да бъде по реда на няколко километра ( km ) за на симулация до около km за на симулация .,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,สำหรับ ของเก่า ที่ จริงจัง ลอง honeychurch ของเก่า ที่ ไม่ สำหรับ เฟอร์นิเจอร์ และ เงิน ไท ร้อง บริษัท ที่ สำหรับ ลาย คราม,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Он увеличил давление .,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"S Jak sobie życzysz: Widzisz, jak Hitler zabija Żydów?",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,Parfait pour mon Acer Aspire ! รับ อะไหล่ ที่ มี ช่องว่าง ใหญ่ ใน ตัว พวกเขา اور میں خیال کرتا ہوں کہ کینیڈی کے بارے میں ایک ایسا خیال ہے کہ میرے خیال میں ایک ایسا لگتا ہے کہ اصل بات یہ ہے ، یا آپ جانتے ہیں کہ آپ اس طرح کی دلچسپی رکھتے ہیں ۔,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
49996,"روندوا دیا گیا ، اس نے گھوڑے پر قبضہ کر لیا ۔ Przetrzymywanych jest około więźniów z krajów, w tym Kanady, wielu schwytanych podczas wojny z siecią Al-Kaida w Afganistanie. La Russia ha ratificato la versione modificata, ma gli Stati Uniti e gli altri membri della NATO si sono rifiutati di farlo fino a quando il governo russo non ritirerà le truppe dalle ex repubbliche sovietiche della Moldavia e della Georgia.",0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0
49997,правни услуги корпорация v. Een groep mensen in boten die naar de wal racen. L'Australia a istituire la più grande riserva marina del mondo,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
49998,"Yol kenarı çöp toplamak için g- Zirvelerde ' i skip . लेकिन मुझे आज रात एक क ् लास मिल गया है इसलिए मैं इन ् हें रास ् ते से बाहर करना चाहता था Heute kam mein Überraschungspaket, wie erwartet. Ich will mich für diesen Preis nicht beklagen, aber es war viel Plunder enthalten, vieles für Weihnachten aber auch einiges, das mir gefallen hat wie zwei Lichterketten und viele Geschenkbänder. Auch die Servietten sind ganz süß, allerdings wie gesagt für Weihnachten. Ich habe euch mal ein Foto hochgeladen, damit ihr einen Überblick kriegt - wie gesagt, das Paket ist ganz ok, würde jedoch nicht wieder bestellen.",0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0


### Create Train/Test Split

In [21]:
!pip install scikit-learn

  Using cached scikit_multilearn-0.2.0-py3-none-any.whl.metadata (6.0 kB)
Using cached scikit_multilearn-0.2.0-py3-none-any.whl (89 kB)


In [12]:
from sklearn.model_selection import train_test_split

In [58]:
X = multi_df['text']
y = multi_df[unique_labels]
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Build and Train Model

### Create Pipeline

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

In [59]:
base_lr = LogisticRegression(solver='lbfgs')
ClassifierChain(base_lr, order='random')
model = Pipeline(steps = [('vectorizer', CountVectorizer()),
                          ('logistic_regression', MultiOutputClassifier(LogisticRegression()))])

model.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('logistic_regression',
                 MultiOutputClassifier(estimator=LogisticRegression()))])

### Test Model
### Get Score Against Test Set

In [60]:
model.score(X_test, y_test)

0.9126380368098159

### Run Some Manual Predictions

In [61]:
print(unique_labels)
model.predict(["Testing an english phrase to see how the model performs", 
               "os chefes de defesa da estónia, letónia, lituânia, alemanha, itália, assinarão o acordo para fornecer pessoal e financiamento",
               "สำหรับ ของเก่า ที่ จริงจัง ลอง honeychurch ของเก่า ที่ ไม่ สำหรับ เฟอร์นิเจอร์ และ เงิน ไท ร้อง",
               "Here is one that has both english and another สำหรับ ของเก่า ที่ จริงจัง ลอง honeychurch ของเก่า ที่ ไม่ สำหรับ เฟอร์นิเจอร์ และ เงิน ไท ร้อง",
               "Tester une phrase aléatoire and Testing an english phrase to see how the model performs"
              ])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])